In [1]:
import pandas as pd
import numpy as np
from ProgSnap2 import ProgSnap2Dataset
from ProgSnap2 import PS2
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path

In [22]:
early_assignments = np.array([439, 487, 492])
late_assignments = np.array([494, 502])

In [37]:
def verify_users(path, is_train, users):
    subdir = 'Train' if is_train else 'Test'
    path = os.path.join(path, subdir)
    ps2 = ProgSnap2Dataset(os.path.join(path, 'Data'))
    early = pd.read_csv(os.path.join(path, 'early.csv'))
    late = pd.read_csv(os.path.join(path, 'late.csv'))
    main_table = ps2.get_main_table()
    student_table = ps2.load_link_table('Subject')
    code_states = ps2.get_code_states_table()
    
    # Verify the early table
    assert early[PS2.SubjectID].isin(users).all()
    assert early[PS2.AssignmentID].isin(early_assignments).all()
    assert 'Label' in early.columns
    assert early['Label'].mean() != 0
    
    # Verify the late table
    assert late[PS2.SubjectID].isin(users).all()
    assert late[PS2.AssignmentID].isin(late_assignments).all()
    assert ('Label' in late.columns) == is_train
    if (is_train):
        assert early['Label'].mean() != 0
        
    # Verify the main table
    assert main_table[PS2.SubjectID].isin(users).all()
    main_table_assignments = early_assignments
    if (is_train):
        main_table_assignments = np.append(main_table_assignments, late_assignments)
    assert main_table[PS2.AssignmentID].isin(main_table_assignments).all()
    
    # Verify student table
    assert student_table[PS2.SubjectID].isin(users).all()
    assert ('X-Grade' in student_table.columns) == is_train
    
    # Verify the code states table
    assert set(code_states[PS2.CodeStateID]) == set(main_table[PS2.CodeStateID])
    
    

In [43]:
PATH = "data/Release/"

last_semester_users = None

for semester in ['S19', 'F19']:
    split = pd.read_csv(os.path.join(PATH, semester, 'split.csv'))
    print(f'Train % for {semester} is: {split["IsTrain"].mean()}')
    print(f'Count for {semester} is: {len(split.index)}')
    for is_train in [True, False]:
        path = os.path.join(PATH, semester)
        print('Verifying: ' + path)
        users = split[split['IsTrain'] == is_train][PS2.SubjectID]
        verify_users(path, is_train, users)
    
    semester_users = split[PS2.SubjectID]
    if last_semester_users is not None:
        assert len(set(last_semester_users).intersection(semester_users)) == 0
    last_semester_users = semester_users

Train % for S19 is: 0.7477203647416414
Count for S19 is: 329
Verifying: data/Release/S19
Verifying: data/Release/S19
Train % for F19 is: 0.7489795918367347
Count for F19 is: 490
Verifying: data/Release/F19
Verifying: data/Release/F19
